## this is from the hugging face implemenation of the Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

<link>https://huggingface.co/blog/fine-tune-whisper</link>

# import modules

In [1]:
import os

In [2]:

model_name_or_path = "openai/whisper-small"
language = "English"
language_abbr = "en"
task = "transcribe"
dataset_name = "Tarakeshwaran/Whisper-train-data"

# load data set

In [3]:
from datasets import load_dataset, DatasetDict

data_dict= DatasetDict()

data_dict["train"] = load_dataset(dataset_name, split="train")
data_dict["test"] = load_dataset(dataset_name, split="test")

print(data_dict)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'start', 'end'],
        num_rows: 80
    })
    test: Dataset({
        features: ['audio', 'text', 'start', 'end'],
        num_rows: 20
    })
})


In [4]:
data_dict = data_dict.remove_columns(["start","end"])

In [5]:
data_dict

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 80
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 20
    })
})

## Prepare Feature Extractor, Tokenizer and Data

In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [7]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

In [8]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

In [9]:
print(data_dict["train"][0])

{'audio': {'path': 'sample-005811.mp3', 'array': array([-3.10192730e-25,  8.27180613e-25, -4.65289095e-24, ...,
       -1.07697597e-05,  2.15554501e-05, -2.88033916e-05]), 'sampling_rate': 16000}, 'text': "in alchemy it's called the soul of the world"}


In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [11]:
data_dict = data_dict.map(prepare_dataset, remove_columns=data_dict.column_names["train"], num_proc=1)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [12]:
data_dict["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 80
})

### loading a pretrained checkpoint

In [14]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)

In [15]:
model.generation_config.language = language
model.generation_config.task = task

model.generation_config.forced_decoder_ids = None


### define data collectors

In [16]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [17]:
# initilising the data
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### evaluaion metrice

In [18]:
import evaluate

metric = evaluate.load("wer")

In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### define training arguments

In [20]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


d:\Work\vs_code_doc\repo\Finetuning-whisper-model-with-PRFT-and-LoRA-int-8\env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


C:\Users\tarak\AppData\Local\Temp\ipykernel_2120\3975393406.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [22]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


KeyboardInterrupt: 